In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
%matplotlib qt5

Using TensorFlow backend.


In [2]:
raan = pd.read_csv('xz.csv')
raan

,Unnamed: 0,Mean Anomaly
0,0,5.904736e+02
1,1,9.498960e+02
2,2,1.371916e+03
3,3,1.718384e+03
4,4,1.817394e+03
...,...,...
7132,7132,2.567893e+06
7133,7133,2.568262e+06
7134,7134,2.568621e+06
7135,7135,2.568984e+06


In [3]:
raan = raan.drop('Unnamed: 0', axis = 1)

In [4]:
raan

,Mean Anomaly
0,5.904736e+02
1,9.498960e+02
2,1.371916e+03
3,1.718384e+03
4,1.817394e+03
...,...
7132,2.567893e+06
7133,2.568262e+06
7134,2.568621e+06
7135,2.568984e+06


In [5]:
idx = pd.DataFrame(range(len(raan)))

In [6]:
raan = pd.concat([idx, raan], axis = 1)
raan

,0,Mean Anomaly
0,0,5.904736e+02
1,1,9.498960e+02
2,2,1.371916e+03
3,3,1.718384e+03
4,4,1.817394e+03
...,...,...
7132,7132,2.567893e+06
7133,7133,2.568262e+06
7134,7134,2.568621e+06
7135,7135,2.568984e+06


In [7]:
x = np.array(raan[0])
y = np.array(raan['Mean Anomaly'])
X_train = x[:int(len(x) * 0.8)]
y_train = x[:int(len(x) * 0.8)]
X_test = x[int(len(x) * 0.8):]
y_test = x[int(len(x) * 0.8):]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5709,)
(1428,)
(5709,)
(1428,)


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import keras.backend as K

In [9]:
batch_size = 50
lr = 0.01
epoch = 500

In [9]:
K.clear_session()
model = Sequential()
model.add(Dense(100, activation = tf.nn.relu, input_shape = [1]))
model.add(Dense(50, activation = tf.nn.relu))
model.add(Dense(25, activation = tf.nn.relu))
model.add(Dense(1))

model.compile(loss = 'mse', optimizer = 'adam')

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               200       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 6,551
Trainable params: 6,551
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', patience = 30, verbose = 1, mode = 'min')

In [12]:
r = model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size, 
              validation_data = (X_test, y_test), verbose = 1, callbacks = [es])

Epoch 1/500
115/115 [==============================] - 0s 4ms/step - loss: 1983641.8750 - val_loss: 35.6863
Epoch 2/500
115/115 [==============================] - 0s 2ms/step - loss: 0.5691 - val_loss: 0.0700
Epoch 3/500
115/115 [==============================] - 0s 3ms/step - loss: 0.2096 - val_loss: 0.0068
Epoch 4/500
115/115 [==============================] - 0s 2ms/step - loss: 0.0445 - val_loss: 0.0779
Epoch 5/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0365 - val_loss: 0.0882
Epoch 6/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0364 - val_loss: 0.0513
Epoch 7/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0363 - val_loss: 0.0733
Epoch 8/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0360 - val_loss: 0.0679
Epoch 9/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0358 - val_loss: 0.0515
Epoch 10/500
115/115 [==============================] - 0s 3ms/step - loss: 0.0356 -

In [13]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size = 50)
print(loss_and_metrics)

29/29 [==============================] - 0s 2ms/step - loss: 0.0353
0.03531578183174133


In [14]:
yhat = model.predict(X_test)
yhat2 = model.predict(X_train)

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse1 = sqrt(mean_squared_error(y_test, yhat))
print("Test RMSE: %.5f" % rmse1)

rmse2 = sqrt(mean_squared_error(y_train, yhat2))
print("Train RMSE: %.5f" % rmse2)

Test RMSE: 0.18793
Train RMSE: 0.15505
